# Absorbing State Probability

Write a function solution(m) that takes an array of array of nonnegative ints representing how many times that state has gone to the next state and return an array of ints for each terminal state giving the exact probabilities of each terminal state, represented as the numerator for each state, then the denominator for all of them at the end and in simplest form. The matrix is at most 10 by 10. It is guaranteed that no matter what the current state is, there is a path from that state to a terminal state. That is, the processing will always eventually end in a stable state. The sequence starts in state 0. The denominator will fit within a signed 32-bit integer during the calculation, as long as the fraction is simplified regularly.
<br>
For example, consider the matrix m:
<br>
```python
[
  [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
  [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
  [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
  [0,0,0,0,0,0],  # s3 is terminal
  [0,0,0,0,0,0],  # s4 is terminal
  [0,0,0,0,0,0],  # s5 is terminal
]
```
So, we can consider different paths to terminal states, such as:
<br>
s0 -> s1 -> s3
<br>
s0 -> s1 -> s0 -> s1 -> s0 -> s1 -> s4
<br>
s0 -> s1 -> s0 -> s5
<br>
Tracing the probabilities of each, we find that
<br>
s2 has probability 0
<br>
s3 has probability 3/14
<br>
s4 has probability 1/7
<br>
s5 has probability 9/14
<br>
So, putting that together, and making a common denominator, gives an answer in the form of
[s2.numerator, s3.numerator, s4.numerator, s5.numerator, denominator] which is
[0, 3, 2, 9, 14].

In [ ]:
from fractions import Fraction as frac
from fractions import gcd
from functools import reduce


def solution(m):
    # Your code here
    assert len(m) >= 1
    assert len(m) == len(m[0])

    probabilify_matrix(m)

    terminal_states = []
    non_terminal_states = []
    for i, row in enumerate(m):
        if sum(row) == 0:
            temp = row
            terminal_states.append(row)
        else:
            non_terminal_states.append(row)

    if len(terminal_states) == 1:
        return [1, 1]

    Q = get_Q(m)
    R = get_R(m)
    I_minus_Q = subtract_matrices(get_I(Q), Q)
    F = inverse_matrix(I_minus_Q)
    FR = multiply_matrices(F, R)

    prob_of_terminal_states = FR[0]
    common_denominator = 1 / reduce(gcd, prob_of_terminal_states)

    result = []
    for val in prob_of_terminal_states:
        factor = common_denominator / val.denominator
        numerator = val.numerator * factor
        result.append(int(numerator))
    result.append(int(common_denominator))

    return result

# turn matrix values to probabiltiy
def probabilify_matrix(m):
    for row in m:
        denominator = sum(row)
        if not denominator == 0:
            for i in range(len(row)):
                row[i] = frac(row[i], denominator)
        else:
            for i in range(len(row)):
                row[i] = frac(0)

# get submatrix Q
def get_Q(m):
    Q = []
    for i in range(len(m)):
        if not sum(m[i]) == 0:
            temp = []
            for j in range(len(m[i])):
                if not sum(m[j]) == 0:
                    temp.append(m[i][j])
            Q.append(temp)
    return Q

# get submatrix R
def get_R(m):
    R = []
    for i in range(len(m)):
        if not sum(m[i]) == 0:
            temp = []
            for j in range(len(m[0])):
                if sum(m[j]) == 0:
                    temp.append(m[i][j])
            R.append(temp)
    return R

# subtract to 2 matrices
def subtract_matrices(m1, m2):
    result = []
    for i in range(len(m1)):
        temp = []
        for j in range(len(m1[i])):
            sub = m1[i][j] - m2[i][j]
            temp.append(sub)
        result.append(temp)
    return result


# get the identity matrix with row*col of matrix m
def get_I(m):
    I = []
    for i in range(len(m)):
        temp = []
        for j in range(len(m)):
            if j == i:
                temp.append(frac(1))
            else:
                temp.append(frac(0))
        I.append(temp)
    return I

# multiply matrices
def multiply_matrices(m1, m2):
    product = []
    m2_cols = rotate_matrix(m2)
    for x in range(len(m1)):
        temp = []
        for y in range(len(m2_cols)):
            dot_prod = dot_product(m1[x], m2_cols[y])
            temp.append(dot_prod)
        product.append(temp)
    return product

# rotate matrix by 90*
def rotate_matrix(m):
    cols = []
    for c in range(len(m[0])):
        temp = []
        for r in range(len(m)):
            temp.append(m[r][c])
        cols.append(temp)
    return cols


def dot_product(m1, m2):
    dot_prod = 0
    for x, y in zip(m1, m2):
        dot_prod += x * y
    return dot_prod


'''
Here we go.
Inverse matrix m.
m is a square matrix
'''
def inverse_matrix(m):
    assert len(m) == len(m[0])

    # base case 2x2 matrix
    if len(m) == 2:
        coef = frac(1, det(m))
        inversed_matrix = [[coef * m[1][1], (-1) * coef * m[0][1]],
                           [coef * (-1) * m[1][0], coef * m[0][0]]]
        return inversed_matrix

    matrix_of_minors = get_matrix_of_minors(m)
    matrix_of_cofactors = get_matrix_of_cofactors(matrix_of_minors)
    adjugated_matrix = adjugate_matrix(matrix_of_cofactors)

    det_m = det(m)
    inversed_matrix = multiply_matrix_by_scalar(adjugated_matrix, frac(1, det_m))

    return inversed_matrix

# multiply matrix by scalar. * note that scalar is a Python Fraction
def multiply_matrix_by_scalar(m, frac_num):
    for i in range(len(m)):
        for j in range(len(m[0])):
            m[i][j] = frac(m[i][j]) * frac_num
    return m

# get adjoint matrix
def adjugate_matrix(m):
    x = 0
    for i in range(len(m)):
        for j in range(x, len(m[0])):
            m[i][j], m[j][i] = m[j][i], m[i][j]
        x += 1
    return m

# get matrix of cofactors
def get_matrix_of_cofactors(m):
    for i in range(len(m)):
        for j in range(len(m[0])):
            m[i][j] *= (-1)**(i+j)
    return m

# get the matrix of minors of matrix
def get_matrix_of_minors(m):
    matrix_of_minors = []
    for i in range(len(m)):
        temp = []
        for j in range(len(m[0])):
            temp.append(det(get_submatrix(m, i, j)))
        matrix_of_minors.append(temp)
    return matrix_of_minors

# find the determinate of a matrix
def det(m):
    # base case 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1] - m[0][1]*m[1][0]

    det_val = 0
    for j in range(len(m[0])):
        temp = m[0][j] * det(get_submatrix(m, 0, j))
        det_val += ((-1)**j)*temp
    return det_val

# get submatrix with a given row and column
def get_submatrix(m, r, c):
    sub_m = []
    for row in (m[:r] + m[r + 1:]):
        temp = row[:c] + row[c + 1:]
        sub_m.append(temp)
    return sub_m